In [2]:
from time import time
import spreg, libpysal
import pandas as pd

In [ ]:
def w_workbench(w, y, x, c, reps=1):
    timing = pd.DataFrame()
    
    for rep in range(reps):
    
        # Spreg #
        s = time()
        m1 = spreg.GM_Combo_Het(y, x, w)
        timing.loc["GM_Combo_Het", f"time_{rep}"] = time() - s

        s = time()
        m2 = spreg.ML_Lag(y, x, w)
        timing.loc["ML_Lag", f"time_{rep}"] = time() - s

        # Lags
        s = time()
        ls = libpysal.weights.lag_spatial(w, y)
        timing.loc["lag_spatial", f"time_{rep}"] = time() - s

        s = time()
        lc = libpysal.weights.lag_categorical(w, c)
        timing.loc["lag_categorical", f"time_{rep}"] = time() - s

        # Neighbor query #
        s = time()
        nq = neighbor_query(w)
        timing.loc["neighbor_query", f"time_{rep}"] = time() - s

        # Components
        s = time()
        components = w.component_labels
        timing.loc["components", f"time_{rep}"] = time() - s

        # Higher order (exclusive and inclusive)
        s = time()
        w10 = libpysal.weights.higher_order(w, k=10)
        timing.loc["higher_order", f"time_{rep}"] = time() - s
        s = time()
        w10inc = libpysal.weights.higher_order(w, k=10, lower_order=True)
        timing.loc["higher_order_inclusive", f"time_{rep}"] = time() - s
        
    timing['mean'] = timing.mean(axis=1)
    timing['std'] = timing.iloc[:, :reps].std(axis=1)
    
    return {
        "timing": timing,
        "m1": m1,
        "m2": m2,
        "ls": ls,
        "lc": lc,
        "nq": nq,
        "components": components,
        "w10": w10
        "w10inc": w10inc
    }

def neighbor_query(w):
    d = {}
    for i in range(w.n):
        d[i] = w.neighbors[i]
    return d